# 🎯 Notebook 03 - Latihan & Tugas Praktikum

**Selamat datang di Notebook 03!** Ini adalah notebook terakhir di mana Anda akan menerapkan semua yang sudah dipelajari dalam sebuah **tugas praktikum**.

---

## 🎯 Tujuan Pembelajaran

Setelah menyelesaikan notebook ini, Anda akan dapat:
1. **Mengintegrasikan** semua skill dari Notebook 01 & 02
2. **Mengagregasi** data geospasial per wilayah
3. **Membuat marker cluster** untuk banyak titik
4. **Menyusun insight** dari visualisasi peta
5. **Menyelesaikan tugas** dengan rubrik penilaian

⏱️ **Estimasi waktu:** 60-90 menit

---

## 📋 TUGAS PRAKTIKUM

### Deskripsi Tugas

**Buat peta interaktif yang menampilkan:**
1. Persebaran UMKM di Jawa Timur
2. Agregasi jumlah UMKM per kecamatan (tabel)
3. Marker cluster untuk mengelompokkan titik yang berdekatan
4. Popup informatif untuk setiap marker
5. Insight/narasi singkat tentang pola sebaran UMKM

---

### 📊 Rubrik Penilaian (Total: 100 poin)

| Aspek | Kriteria | Poin |
|-------|---------|------|
| **Data** | Data dimuat dengan benar, tidak ada error | 15 |
| | Agregasi per kecamatan dilakukan dengan tepat | 15 |
| **Peta** | Peta interaktif ditampilkan dengan baik | 15 |
| | Marker cluster berfungsi | 10 |
| | Popup/tooltip informatif & rapi | 15 |
| **Insight** | Narasi menjelaskan pola sebaran | 15 |
| | Insight didukung data (angka, persentase) | 10 |
| **Kode** | Kode bersih, ada komentar | 5 |

---

## 🛠️ BAGIAN 1: Persiapan Data

In [ ]:
# Import library yang diperlukan
import pandas as pd
import geopandas as gpd
import folium
from folium.plugins import MarkerCluster
from pathlib import Path

# Setup path
ROOT = Path('..') if Path('.').name == 'notebooks' else Path('.')
CSV_UMKM = ROOT / 'data' / 'raw' / 'umkm_sample.csv'

# Load data
df = pd.read_csv(CSV_UMKM)

print(f'✅ Data UMKM berhasil dimuat: {len(df)} baris\n')
print('📋 Preview data:')
df.head()

---

## 📊 BAGIAN 2: Eksplorasi & Agregasi Data

### Langkah 1: Analisis Deskriptif

Sebelum membuat peta, kita perlu **memahami data** dulu:

In [ ]:
# Statistik dasar
print('📈 STATISTIK DASAR\n')
print(f'Total UMKM: {len(df)}')
print(f'Rata-rata skor: {df["skor"].mean():.2f}')
print(f'Skor tertinggi: {df["skor"].max()}')
print(f'Skor terendah: {df["skor"].min()}\n')

# Distribusi per jenis
print('🏷️ DISTRIBUSI PER JENIS UMKM\n')
jenis_count = df['jenis'].value_counts()
print(jenis_count)
print(f'\nJenis terbanyak: {jenis_count.index[0]} ({jenis_count.iloc[0]} UMKM)')

### Langkah 2: Agregasi Per Kecamatan

Ini adalah bagian penting! Kita akan menghitung **jumlah UMKM** dan **rata-rata skor** per kecamatan:

In [ ]:
# Agregasi per kecamatan
agg_kecamatan = df.groupby('kecamatan').agg({
    'id': 'count',           # jumlah UMKM
    'skor': 'mean',          # rata-rata skor
    'lat': 'mean',           # koordinat pusat (untuk marker)
    'lon': 'mean'
}).rename(columns={'id': 'jumlah_umkm', 'skor': 'rata_rata_skor'})

# Urutkan berdasarkan jumlah
agg_kecamatan = agg_kecamatan.sort_values('jumlah_umkm', ascending=False)

print('📍 AGREGASI PER KECAMATAN\n')
print(agg_kecamatan)
print(f'\n✅ Kecamatan dengan UMKM terbanyak: {agg_kecamatan.index[0]} ({agg_kecamatan.iloc[0]["jumlah_umkm"]} UMKM)')

> **💡 PENJELASAN AGREGASI:**
> - `groupby('kecamatan')` → kelompokkan data berdasarkan kecamatan
> - `agg({'id': 'count'})` → hitung jumlah baris per grup
> - `'skor': 'mean'` → hitung rata-rata skor
> - Koordinat rata-rata berguna untuk menempatkan marker di "pusat" kecamatan

---

## 🗺️ BAGIAN 3: Membuat Peta Interaktif

### Langkah 1: Peta Dasar dengan Marker Cluster

**Marker Cluster** mengelompokkan titik yang berdekatan menjadi satu cluster. Saat di-zoom, cluster akan "pecah" jadi marker individual.

**Kenapa penting?** Jika punya ratusan/ribuan marker, peta jadi lambat & berantakan. Cluster membuat tampilan lebih bersih!

In [ ]:
# Buat peta dasar
center_lat = df['lat'].mean()
center_lon = df['lon'].mean()

m = folium.Map(
    location=[center_lat, center_lon],
    zoom_start=7,
    tiles='CartoDB positron'
)

# Buat marker cluster
marker_cluster = MarkerCluster(
    name='UMKM Cluster',
    overlay=True,
    control=True
).add_to(m)

print('✅ Peta dasar dan cluster berhasil dibuat!')
print('Sekarang akan menambahkan marker...')

### Langkah 2: Tambahkan Marker ke Cluster

Kita akan membuat popup yang **informatif** dengan data lengkap:

In [ ]:
# Palet warna untuk jenis UMKM
palette = {
    'makanan': '🍽️ red',
    'kue': '🎂 purple',
    'jasa': '🔧 blue',
    'kerajinan': '🎨 green',
    'minuman': '☕ orange',
    'retail': '🏪 darkgreen',
    'jasaha': '⚙️ gray'
}

# Tambahkan marker untuk setiap UMKM
for idx, row in df.iterrows():
    # Ambil icon dan warna
    jenis_info = palette.get(row['jenis'], '📍 black')
    icon_emoji, color = jenis_info.split(' ')
    
    # Tentukan warna berdasarkan skor
    if row['skor'] >= 80:
        skor_badge = '⭐ Excellent'
        skor_color = 'green'
    elif row['skor'] >= 70:
        skor_badge = '✓ Good'
        skor_color = 'orange'
    else:
        skor_badge = '⚠️ Perlu Perbaikan'
        skor_color = 'red'
    
    # Buat HTML popup yang menarik
    popup_html = f"""
    <div style='font-family: Arial; min-width: 200px;'>
        <h3 style='color: {color}; margin-bottom: 10px;'>
            {icon_emoji} {row['nama']}
        </h3>
        <hr style='margin: 5px 0; border-color: {color};'>
        <table style='width:100%; font-size: 12px;'>
            <tr><td><b>📍 Lokasi:</b></td><td>{row['kota']}</td></tr>
            <tr><td><b>🗺️ Kecamatan:</b></td><td>{row['kecamatan']}</td></tr>
            <tr><td><b>🏷️ Jenis:</b></td><td>{row['jenis'].title()}</td></tr>
            <tr><td><b>⭐ Skor:</b></td><td>
                <span style='background-color: {skor_color}; color: white; padding: 2px 8px; border-radius: 3px;'>
                    {row['skor']}/100
                </span>
            </td></tr>
            <tr><td><b>Status:</b></td><td>{skor_badge}</td></tr>
        </table>
    </div>
    """
    
    # Tambahkan marker ke cluster
    folium.Marker(
        location=[row['lat'], row['lon']],
        popup=folium.Popup(popup_html, max_width=300),
        icon=folium.Icon(color=color, icon='info-sign')
    ).add_to(marker_cluster)

print(f'✅ {len(df)} marker berhasil ditambahkan ke cluster!')

> **💡 TIPS POPUP:**
> - Gunakan **HTML** untuk styling popup yang menarik
> - Tambahkan **emoji** untuk visual lebih menarik
> - Gunakan **tabel** untuk format data yang rapi
> - **Color coding** membantu user cepat memahami status

---

### Langkah 3: Tambahkan Layer Control

Layer Control memungkinkan user untuk toggle (nyalakan/matikan) layer tertentu:

In [ ]:
# Tambahkan layer control
folium.LayerControl().add_to(m)

# Tambahkan judul peta
title_html = '''
<div style="position: fixed; 
     top: 10px; left: 50px; width: 400px; height: 60px; 
     background-color: white; border:2px solid grey; z-index:9999; 
     padding: 10px; border-radius: 5px; box-shadow: 2px 2px 6px rgba(0,0,0,0.3);">
     <h4 style="margin: 0; padding: 0;">📍 Peta Persebaran UMKM Jawa Timur</h4>
     <p style="margin: 5px 0 0 0; font-size: 12px; color: #666;">
         Klik marker untuk detail | Zoom untuk lihat cluster
     </p>
</div>
'''
m.get_root().html.add_child(folium.Element(title_html))

print('✅ Layer control dan judul berhasil ditambahkan!')

### Langkah 4: Simpan & Tampilkan Peta

In [ ]:
# Simpan peta
output_path = ROOT / 'exports' / 'tugas_peta_umkm_cluster.html'
m.save(str(output_path))

print(f'💾 Peta berhasil disimpan ke: {output_path}')
print('🌐 Buka file HTML untuk melihat peta interaktif!\n')
print('🔍 CARA MENGGUNAKAN PETA:')
print('  1. Zoom in → cluster akan pecah jadi marker individual')
print('  2. Klik marker → lihat detail UMKM')
print('  3. Toggle layer di kanan atas untuk kontrol tampilan')

# Tampilkan di Jupyter
m

---

## 📝 BAGIAN 4: Insight & Narasi

### Contoh Analisis & Insight

Berdasarkan data dan visualisasi di atas, mari kita susun **insight**:

In [ ]:
# Hitung beberapa statistik untuk insight
total_umkm = len(df)
jenis_terbanyak = df['jenis'].value_counts().index[0]
jumlah_jenis_terbanyak = df['jenis'].value_counts().iloc[0]
persen_jenis = (jumlah_jenis_terbanyak / total_umkm) * 100

kota_terbanyak = df['kota'].value_counts().index[0]
jumlah_kota_terbanyak = df['kota'].value_counts().iloc[0]
persen_kota = (jumlah_kota_terbanyak / total_umkm) * 100

skor_tinggi = len(df[df['skor'] >= 80])
persen_skor_tinggi = (skor_tinggi / total_umkm) * 100

print('📊 INSIGHT & NARASI\n')
print('='*60)
print('\n🎯 RINGKASAN EKSEKUTIF\n')
print(f'Dataset ini mencakup {total_umkm} UMKM yang tersebar di beberapa kota')
print(f'di Jawa Timur. Berikut adalah temuan utama:\n')

print('1️⃣ DISTRIBUSI JENIS USAHA')
print(f'   • Jenis usaha dominan: {jenis_terbanyak.upper()}')
print(f'   • Jumlah: {jumlah_jenis_terbanyak} UMKM ({persen_jenis:.1f}% dari total)')
print(f'   • Implikasi: Sektor {jenis_terbanyak} berpotensi besar di wilayah ini\n')

print('2️⃣ KONSENTRASI GEOGRAFIS')
print(f'   • Kota dengan UMKM terbanyak: {kota_terbanyak}')
print(f'   • Jumlah: {jumlah_kota_terbanyak} UMKM ({persen_kota:.1f}% dari total)')
print(f'   • Implikasi: {kota_terbanyak} adalah hub ekonomi UMKM\n')

print('3️⃣ KUALITAS UMKM (berdasarkan skor)')
print(f'   • UMKM berkinerja tinggi (skor ≥80): {skor_tinggi} ({persen_skor_tinggi:.1f}%)')
print(f'   • Rata-rata skor keseluruhan: {df["skor"].mean():.1f}/100')
print(f'   • Implikasi: Masih ada ruang perbaikan kualitas UMKM\n')

print('4️⃣ REKOMENDASI')
print('   • Fokus pembinaan di kota dengan skor rendah')
print('   • Diversifikasi jenis usaha di luar sektor dominan')
print('   • Program clustering untuk efisiensi distribusi & logistik')
print('\n' + '='*60)

> **💡 TIPS MENULIS INSIGHT:**
> - **Mulai dengan angka** - berikan konteks kuantitatif
> - **Gunakan persentase** - lebih mudah dipahami daripada angka absolut
> - **Jelaskan implikasi** - "So what?" dari setiap temuan
> - **Berikan rekomendasi** - actionable next steps
> - **Visual + narasi** - peta mendukung cerita Anda

---

## ✍️ TUGAS ANDA

Sekarang giliran Anda! Modifikasi kode di atas dengan:

### Tugas Wajib:
1. ✅ Jalankan semua sel kode di atas tanpa error
2. ✅ Pastikan peta cluster berfungsi dengan baik
3. ✅ Tulis insight Anda sendiri (minimal 3 poin)

### Tugas Tambahan (Opsional, +10 poin bonus):
1. Tambahkan **heatmap** untuk visualisasi densitas UMKM
2. Buat **peta kedua** dengan Plotly untuk perbandingan
3. Ekspor peta ke **PDF** atau **screenshot** untuk presentasi

---

## 📋 Checklist Sebelum Submit

- [ ] Semua sel kode berjalan tanpa error
- [ ] Peta tersimpan di `exports/tugas_peta_umkm_cluster.html`
- [ ] Agregasi per kecamatan benar (cek manual)
- [ ] Popup informatif & rapi
- [ ] Marker cluster berfungsi (coba zoom in/out)
- [ ] Insight ditulis dengan lengkap (minimal 3 poin)
- [ ] Kode diberi komentar yang jelas

---

## 🎉 Selamat!

Anda telah menyelesaikan **semua 3 notebook** dalam modul ini!

### 🎓 Apa yang Sudah Anda Pelajari:

**Notebook 01:**
- ✅ Sistem koordinat (lat/lon)
- ✅ Proyeksi peta (CRS)
- ✅ GeoPandas untuk peta statis
- ✅ Overlay beberapa layer

**Notebook 02:**
- ✅ Folium untuk peta interaktif
- ✅ Plotly untuk scatter mapbox
- ✅ Perbandingan alat visualisasi
- ✅ Kustomisasi popup & tooltip

**Notebook 03:**
- ✅ Agregasi data geospasial
- ✅ Marker clustering
- ✅ Analisis & insight dari peta
- ✅ Project end-to-end

---

## 🚀 Langkah Selanjutnya

Untuk meningkatkan skill visualisasi geospasial Anda:

1. **Eksplorasi Data Riil**
   - Download data dari [Natural Earth](https://www.naturalearthdata.com/)
   - Gunakan data [OpenStreetMap](https://www.openstreetmap.org/)
   - Cari dataset di [Data.go.id](https://data.go.id/)

2. **Pelajari Fitur Lanjutan**
   - **Folium:** HeatMap, Choropleth, TimeSliderChoropleth
   - **GeoPandas:** Spatial joins, buffer, overlay analysis
   - **Plotly Dash:** Dashboard interaktif real-time

3. **Tools Lanjutan**
   - **QGIS:** Analisis spasial kompleks
   - **Kepler.gl:** Visualisasi big data 3D
   - **Mapbox GL JS:** Custom web maps

4. **Project Ideas**
   - Peta persebaran COVID-19
   - Analisis aksesibilitas rumah sakit
   - Heatmap kejahatan kota
   - Routing optimization

---

## 📚 Referensi Tambahan

- **GeoPandas Docs:** https://geopandas.org/
- **Folium Docs:** https://python-visualization.github.io/folium/
- **Plotly Docs:** https://plotly.com/python/
- **QGIS Tutorials:** https://www.qgistutorials.com/

---

> **💬 Feedback:**
> Modul ini dibuat untuk pembelajaran. Jika ada saran perbaikan atau menemukan error, silakan diskusikan dengan instruktur.

---

**Terima kasih & selamat belajar! 🎓🗺️**